In [36]:
import os
import cv2
import numpy as np

# Configuration
output_dir = "digits_dataset"
image_size = 28
font_scale = 1.0
font_thickness = 2

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Map OpenCV font faces to corresponding fonts
fonts = [
    cv2.FONT_HERSHEY_COMPLEX,
    cv2.FONT_HERSHEY_SIMPLEX
]

# Function to generate digit images
def generate_digit_images():
    data = []
    labels = []
    
    for cv2_font in fonts:
        for digit in range(10):
            # Create a black background
            image = np.zeros((image_size, image_size), dtype=np.uint8)

            # Put the digit in the center of the image
            text = str(digit)
            text_size = cv2.getTextSize(text, cv2_font, font_scale, font_thickness)[0]
            text_x = (image_size - text_size[0]) // 2
            text_y = (image_size + text_size[1]) // 2
            
            cv2.putText(image, text, (text_x, text_y), cv2_font, font_scale, 255, thickness=font_thickness)
            
            # Append image data and label
            data.append(image)
            labels.append(digit)

    # Convert to numpy arrays
    X = np.array(data)
    y = np.array(labels)
    
    return X, y

In [37]:
from digits_recognition.experimentation.dataset.save_pickle_data import save_pickle_data

X, y = generate_digit_images()

save_pickle_data('test.pkl', X, y)

In [38]:
from digits_recognition.experimentation.modeling.evaluate import setup_components
import yaml

with open('../params.yaml', 'r') as file:
    params = yaml.safe_load(file)

MODEL_PATH = '../models/digit_classifier.pth'
TEST_SET_PATH = 'test.pkl'
BATCH_SIZE = 64
RANDOM_SEED = 42
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1
CLASS_COUNT = 10

model, device, loader, _ = setup_components(
    TEST_SET_PATH,
    BATCH_SIZE,
    MODEL_PATH,
    IMAGE_HEIGHT,
    IMAGE_WIDTH,
    IMAGE_CHANNELS,
    CLASS_COUNT,
    RANDOM_SEED
)

In [39]:
for image, _ in loader:
    image = image.to(device)
    model(image)